In [2]:
import pandas as pd
import numpy as np
import convertCDF as cdf
import createModelicaCode as cmc
import PowerToolbox as ptbx
import os
import pypower.api as pyp
import pypower.opf_args as opf_args
from pypower.ext2int import ext2int

from IPython.core.display import display, HTML
pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

In [6]:
pti = False
#ieee folder
cdfname = "ieee14cdf"
toModelica = cdf.createModelicaData(cdfname)

In [7]:
busCols = ["Bus No.","Area","Zone","Type","Voltage p.u.","Angle","PL","QL","PG","QG","Voltage (base)","Voltage (desired)","Max.","Min.","G p.u.","B p.u.","Remote Control"]
busFrame = pd.DataFrame.from_records(toModelica[1],columns = busCols)
print ("Types: (0): PQ (1): PQ (limited [V]) (2): PV (limited [VAR]) (3): Slack")
busFrame


Types: (0): PQ (1): PQ (limited [V]) (2): PV (limited [VAR]) (3): Slack


,Bus No.,Area,Zone,Type,Voltage p.u.,Angle,PL,QL,PG,QG,Voltage (base),Voltage (desired),Max.,Min.,G p.u.,B p.u.,Remote Control
0,1,1,1,3,1.060,0.0,0.0,0.0,232.4,-16.9,0.0,1.060,0.0,0.0,0.0,0.0,0
1,2,1,1,2,1.045,-4.98,21.7,12.7,40.0,42.4,0.0,1.045,50.0,-40.0,0.0,0.0,0
2,3,1,1,2,1.010,-12.72,94.2,19.0,0.0,23.4,0.0,1.010,40.0,0.0,0.0,0.0,0
3,4,1,1,0,1.019,-10.33,47.8,-3.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,1,1,0,1.020,-8.78,7.6,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,6,1,1,2,1.070,-14.22,11.2,7.5,0.0,12.2,0.0,1.070,24.0,-6.0,0.0,0.0,0
6,7,1,1,0,1.062,-13.37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7,8,1,1,2,1.090,-13.36,0.0,0.0,0.0,17.4,0.0,1.090,24.0,-6.0,0.0,0.0,0
8,9,1,1,0,1.056,-14.94,29.5,16.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.19,0
9,10,1,1,0,1.051,-15.10,9.0,5.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [8]:
branchCols = ["From","To","Area","Zone","Circuit No.","Type","R p.u.","X p.u.","B p.u.","MVA1","MVA2","MVA3","Control Bus","Side","Tr. Ratio","Tr. Angle","Tap/Phase Min.","Tap/Phase Max.","Step Size","Min.","Max."]
branchFrame = pd.DataFrame.from_records(toModelica[2], columns = branchCols)
branchFrame

,From,To,Area,Zone,Circuit No.,Type,R p.u.,X p.u.,B p.u.,MVA1,MVA2,MVA3,Control Bus,Side,Tr. Ratio,Tr. Angle,Tap/Phase Min.,Tap/Phase Max.,Step Size,Min.,Max.
0,1,2,1,1,1,0,0.01938,0.05917,0.0528,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,5,1,1,1,0,0.05403,0.22304,0.0492,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,1,1,1,0,0.04699,0.19797,0.0438,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,4,1,1,1,0,0.05811,0.17632,0.0340,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,5,1,1,1,0,0.05695,0.17388,0.0346,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3,4,1,1,1,0,0.06701,0.17103,0.0128,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4,5,1,1,1,0,0.01335,0.04211,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4,7,1,1,1,0,0.0,0.20912,0.0,0,0,0,0,0,0.978,0.0,0.0,0.0,0.0,0.0,0.0
8,4,9,1,1,1,0,0.0,0.55618,0.0,0,0,0,0,0,0.969,0.0,0.0,0.0,0.0,0.0,0.0
9,5,6,1,1,1,0,0.0,0.25202,0.0,0,0,0,0,0,0.932,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
config = ptbx.configPara() 
config.pti = True
config.dynamic = True
config.Leistung = False
config.Soft = False
config.Modus = 2
config.opt = 2
#case folder
caseName = "case14"
bxObj = ptbx.InitCases(caseName)
toModelica = []
toModelica.append("")
toModelica.append(bxObj.busdata)
toModelica.append(bxObj.branchdata)

pfObj = cmc.PowerFlow(toModelica,bxObj.System,config)

In [10]:
KAM = pd.SparseDataFrame([ pd.SparseSeries(pfObj.Y[i].toarray().ravel()) 
                              for i in np.arange(pfObj.Y.shape[0]) ])

KAMF = pd.SparseDataFrame([ pd.SparseSeries(pfObj.Yf[i].toarray().ravel()) 
                              for i in np.arange(pfObj.Yf.shape[0]) ])

KAM


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,(6.025029055768224-19.447070205514382j),(-4.999131600798035+15.263086523179553j),0j,0j,(-1.025897454970189+4.234983682334831j),0j,0j,0j,0j,0j,0j,0j,0j,0j
1,(-4.999131600798035+15.263086523179553j),(9.521323610814779-30.272115398779064j),(-1.1350191923073958+4.781863151757718j),(-1.686033150614943+5.115838325872083j),(-1.7011396670944048+5.193927397969713j),0j,0j,0j,0j,0j,0j,0j,0j,0j
2,0j,(-1.1350191923073958+4.781863151757718j),(3.1209949022329564-9.82238012935164j),(-1.9859757099255606+5.0688169775939205j),0j,0j,0j,0j,0j,0j,0j,0j,0j,0j
3,0j,(-1.686033150614943+5.115838325872083j),(-1.9859757099255606+5.0688169775939205j),(10.512989522036175-38.654171207607796j),(-6.840980661495672+21.578553981691588j),0j,4.889512660317341j,0j,1.8554995578159006j,0j,0j,0j,0j,0j
4,(-1.025897454970189+4.234983682334831j),(-1.7011396670944048+5.193927397969713j),0j,(-6.840980661495672+21.578553981691588j),(9.568017783560265-35.533639456044824j),4.257445335253384j,0j,0j,0j,0j,0j,0j,0j,0j
5,0j,0j,0j,0j,4.257445335253384j,(6.579923407466222-17.34073280991911j),0j,0j,0j,0j,(-1.9550285631772604+4.0940743442404415j),(-1.525967440450974+3.1759639650294003j),(-3.0989274038379877+6.102755448193116j),0j
6,0j,0j,0j,4.889512660317341j,0j,0j,-19.549005948264654j,5.676979846721544j,9.09008271975275j,0j,0j,0j,0j,0j
7,0j,0j,0j,0j,0j,0j,5.676979846721544j,-5.676979846721544j,0j,0j,0j,0j,0j,0j
8,0j,0j,0j,1.8554995578159006j,0j,0j,9.09008271975275j,0j,(5.326055039467359-24.092506375267877j),(-3.902049552447428+10.365394127060915j),0j,0j,0j,(-1.4240054870199312+3.0290504569306034j)
9,0j,0j,0j,0j,0j,0j,0j,0j,(-3.902049552447428+10.365394127060915j),(5.782934306147828-14.768337876521436j),(-1.8808847537003996+4.402943749460521j),0j,0j,0j


In [11]:
pfObj.YDense

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,(6.025029055768224-19.447070205514382j),(-4.999131600798035+15.263086523179553j),0j,0j,(-1.025897454970189+4.234983682334831j),0j,0j,0j,0j,0j,0j,0j,0j,0j
1,(-4.999131600798035+15.263086523179553j),(9.521323610814779-30.272115398779064j),(-1.1350191923073958+4.781863151757718j),(-1.686033150614943+5.115838325872083j),(-1.7011396670944048+5.193927397969713j),0j,0j,0j,0j,0j,0j,0j,0j,0j
2,0j,(-1.1350191923073958+4.781863151757718j),(3.1209949022329564-9.82238012935164j),(-1.9859757099255606+5.0688169775939205j),0j,0j,0j,0j,0j,0j,0j,0j,0j,0j
3,0j,(-1.686033150614943+5.115838325872083j),(-1.9859757099255606+5.0688169775939205j),(10.512989522036175-38.654171207607796j),(-6.840980661495672+21.578553981691588j),0j,4.889512660317341j,0j,1.8554995578159006j,0j,0j,0j,0j,0j
4,(-1.025897454970189+4.234983682334831j),(-1.7011396670944048+5.193927397969713j),0j,(-6.840980661495672+21.578553981691588j),(9.568017783560265-35.533639456044824j),4.257445335253384j,0j,0j,0j,0j,0j,0j,0j,0j
5,0j,0j,0j,0j,4.257445335253384j,(6.579923407466222-17.34073280991911j),0j,0j,0j,0j,(-1.9550285631772604+4.0940743442404415j),(-1.525967440450974+3.1759639650294003j),(-3.0989274038379877+6.102755448193116j),0j
6,0j,0j,0j,4.889512660317341j,0j,0j,-19.549005948264654j,5.676979846721544j,9.09008271975275j,0j,0j,0j,0j,0j
7,0j,0j,0j,0j,0j,0j,5.676979846721544j,-5.676979846721544j,0j,0j,0j,0j,0j,0j
8,0j,0j,0j,1.8554995578159006j,0j,0j,9.09008271975275j,0j,(5.326055039467359-24.092506375267877j),(-3.902049552447428+10.365394127060915j),0j,0j,0j,(-1.4240054870199312+3.0290504569306034j)
9,0j,0j,0j,0j,0j,0j,0j,0j,(-3.902049552447428+10.365394127060915j),(5.782934306147828-14.768337876521436j),(-1.8808847537003996+4.402943749460521j),0j,0j,0j


In [18]:
#case folder
ptiname = "casePrimer"
config = ptbx.configPara()
config.pti = True
config.Soft = False
config.Modus=2
pySys = ptbx.InitCases(ptiname)
toModelica=[]
toModelica.append("")
toModelica.append(pySys.busdata)
toModelica.append(pySys.branchdata)

pfObj = cmc.PowerFlow(toModelica,pySys.System,config)
KAM = pfObj.getYfromPyPower()

In [21]:
busColsPTI = ["Bus No.","Type","Pd","Qd","Gs","Bs","Area","Vm","Va","basekV","zone","Vmmax","Vmmin"]
busFramePTI = pd.DataFrame.from_records(pySys.busdata,columns = busColsPTI)
busFramePTI

,Bus No.,Type,Pd,Qd,Gs,Bs,Area,Vm,Va,basekV,zone,Vmmax,Vmmin
0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,100.0,1.0,1.00,1.00
1,2.0,1.0,0.0,0.0,0.0,30.0,1.0,1.0,0.0,100.0,1.0,1.05,0.95
2,3.0,2.0,0.0,0.0,5.0,0.0,1.0,1.0,0.0,100.0,1.0,1.05,0.95
3,4.0,2.0,90.0,40.0,0.0,0.0,1.0,1.0,0.0,100.0,1.0,1.05,0.95
4,5.0,1.0,23.9,12.9,0.0,0.0,1.0,1.0,0.0,100.0,1.0,1.05,0.95


In [22]:
branchColsPTI = ["From","To","R","X","B","Rating long term","Rating short term","Rating emergency","ratio","angle","status","Anglediff_min","Anglediff_max"]
branchFrame = pd.DataFrame.from_records(pySys.branchdata,columns = branchColsPTI)
branchFrame

,From,To,R,X,B,Rating long term,Rating short term,Rating emergency,ratio,angle,status,Anglediff_min,Anglediff_max
0,1.0,2.0,0.000,0.300,0.00,0.0,0.0,0.0,0.00,0.0,1.0,-180.0,180.0
1,1.0,3.0,0.023,0.145,0.04,0.0,0.0,0.0,0.00,0.0,1.0,-180.0,180.0
2,2.0,4.0,0.006,0.032,0.01,0.0,0.0,0.0,0.00,0.0,1.0,-180.0,180.0
3,3.0,4.0,0.020,0.260,0.00,0.0,0.0,0.0,0.00,-3.0,1.0,-180.0,180.0
4,3.0,5.0,0.000,0.320,0.00,0.0,0.0,0.0,0.98,0.0,1.0,-180.0,180.0
5,4.0,5.0,0.000,0.500,0.00,0.0,0.0,0.0,0.00,0.0,1.0,-180.0,180.0


In [23]:
genFrameColPTI =["No","Pg","Qg","Qmax","Qmin","Vg","baseMVA","status","Pmax","Pmin","Pc1","Pc2","Qc1min","Qc1max","Qc2min","Qc2max","ramp[MW/min]","ramp[MW/10min]","ramp[MW/30min]","ramp[MVar/min]","APF"]
genFrame = pd.DataFrame.from_records(pySys.gen,columns = genFrameColPTI)
genFrame

,No,Pg,Qg,Qmax,Qmin,Vg,baseMVA,status,Pmax,Pmin,Pc1,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp[MW/min],ramp[MW/10min],ramp[MW/30min],ramp[MVar/min],APF
0,1,0,0,99999,-99999,1,100,1,99999,-99999,0,0,0,0,0,0,0,0,0,0,0
1,3,0,0,30,-20,1,100,1,40,10,0,0,0,0,0,0,0,0,0,0,0
2,4,0,0,20,-20,1,100,1,40,5,0,0,0,0,0,0,0,0,0,0,0


Modell 1: Stückweise lineare Kostenfunkiton K = f((po,f0),...,(pn,fn))

Modell 2: Polynomiale Kostenfunktion: K = cn*p^n + ...+ c1*p + c0

In [25]:
costFrameColPTI=["model","startup","shutdown","N","1","2","3"]
costFrame = pd.DataFrame.from_records(pySys.gencost, columns = costFrameColPTI)
costFrame

,model,startup,shutdown,N,1,2,3
0,2.0,0.0,0.0,3.0,0.0,0.35,0.0
1,2.0,0.0,0.0,3.0,0.4,0.20,0.0
2,2.0,0.0,0.0,3.0,0.5,0.30,0.0
